In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rn
from numpy import linalg as LA
from sklearn.model_selection import train_test_split
from scipy import linalg
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.io import loadmat
from collections import Counter
from sklearn.cluster import KMeans
import time
%matplotlib inline


In [2]:
import json
with open('feature_data.json', 'r') as f:
    features = json.load(f)

features_ar = np.array(features)

# scaler = StandardScaler()
# scaler.fit(features_ar)
# features_ar = scaler.transform(features_ar)

In [3]:
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['train_idx'].flatten()
camIds = loadmat('cuhk03_new_protocol_config_labeled.mat')['camId'].flatten()
filelists = loadmat('cuhk03_new_protocol_config_labeled.mat')['filelist'].flatten()
gallery_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['gallery_idx'].flatten()
labels = loadmat('cuhk03_new_protocol_config_labeled.mat')['labels'].flatten()
query_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')['query_idx'].flatten()

train_idxss = np.subtract(train_idxs, 1)
gallery_idxss = np.subtract(gallery_idxs, 1)
query_idxss = np.subtract(query_idxs, 1)

In [4]:
camIds_train = camIds[train_idxss]
filelists_train = filelists[train_idxss]
labels_train = labels[train_idxss]
features_train = features_ar[train_idxss]

camIds_gal = camIds[gallery_idxss]
filelists_gal = filelists[gallery_idxss]
labels_gal = labels[gallery_idxss]
features_gal = features_ar[gallery_idxss]

camIds_query = camIds[query_idxss]
filelists_query = filelists[query_idxss]
labels_query = labels[query_idxss]
features_query = features_ar[query_idxss]

In [5]:
def rankKeval(labels_neigh, label_query):
    contained = False
    for i in range (0, len(labels_neigh)):
        if (labels_neigh[i] == label_query):
            contained = True
    return(contained)

In [6]:
def ret_NN(knn, query_l, query_f, query_Id, k, s_net):
    n = k + s_net
    neigh = knn.kneighbors([query_f], n, return_distance=False) 
    neigh = neigh[0]
    m = 0
    while m<len(neigh):
        if ((query_l == labels_gal[neigh[m]]) and (query_Id == camIds_gal[neigh[m]])):
            neigh = np.delete(neigh, [m])
            m = m-1
        m = m+1
    m = len(neigh)
    if m>k:
        while m>k:
            neigh = np.delete(neigh, m-1)
            m = len(neigh)       
    labels_neigh = [labels_gal[i] for i in neigh]
    return (labels_neigh)

In [7]:
def scores_query(ranklist, query_l, instances, k):
    acc = 0
    TP = 0
    FP = 0
    prec = 0
    rec = 0
    
    if rankKeval(ranklist, query_l) == True:
        acc = 1
        
    for i in range (0, k):
        if (ranklist[i] == query_l):
            TP = TP + 1
        else:
            FP = FP +1
    prec = TP/(TP + FP)
    rec = TP/instances
    
    return(acc, prec, rec)

In [8]:
def scores_k(acc_l, prec_l, rec_l, k, query_n):
    acc_avg = []
    prec_avg = []
    rec_avg = []
    acc_avg.append(sum(acc_l)/query_n)
    prec_avg.append(sum(prec_l)/query_n)
    rec_avg.append(sum(rec_l)/query_n)
    return(acc_avg, prec_avg, rec_avg)

In [9]:
def average(lst, lstlen):
#     print('lst = ', lst)
#     print('lst[0][1] = ', lst[1][0])
#     print('lstlen = ', lstlen)
    avg = 0
    for i in range(0, lstlen-1):
#         print('i = ', i)
#         print('lst[i][0] = ', lst[i][0])
        avg = avg + lst[i][0]
    avg = avg/lstlen
    return(avg)

In [10]:
def build_kNN(featurese, labelse, k, s_net):
    nn= k + s_net
    knn = KNeighborsClassifier(n_neighbors=nn, p=2)
    knn.fit(featurese, labelse)
    return(knn)

In [11]:
def label_instances(labels):
    labels_copy = np.copy(labels)
    set_l = labels_copy.tolist()
    set_l = set(set_l)
    set_l = list(set_l)
    classes_n = len(set_l)
    instances = [0] * 1467
    for i in range(0, classes_n):
        m = 0
        while m<len(labels_copy):
            if (labels_copy[m] == set_l[i]):
                instances[set_l[i]] = instances[set_l[i]] + 1
                labels_copy = np.delete(labels_copy, m, 0)
                m = m - 1
            m = m + 1
    return(instances)

In [12]:
def do_kNN(featurese_gal, labels_gal, featurese_query, labels_query, camIds_query, k, s_net):
    knn = build_kNN(featurese_gal, labels_gal, k, s_net)
    query_n = len(labels_query)
    instances = label_instances(labels_gal)
    acc = [0] * (query_n * k)
    prec = [0] * (query_n * k)
    rec = [0] * (query_n * k)
    AA = [0] * k
    AR = [0] * k 
    AP = [0] * k
    mAP = [0] * k 
    for i in range (0, query_n):
        query_l = labels_query[i]
        query_f = featurese_query[i]
        query_Id = camIds_query[i]
        neigh = ret_NN(knn, query_l, query_f, query_Id, k, s_net)
        for z in range (0, k):
            ind = i+(z*query_n)
            acc[ind], prec[ind], rec[ind] = scores_query(neigh, query_l, instances[query_l], (k-z))
            neigh.pop()
    for i in range (0, k):
        LB = i*query_n
        UB = (i+1)*query_n
        AA[i], AP[i], AR[i] = scores_k(acc[LB:UB], prec[LB:UB], rec[LB:UB], i+1, query_n)
    mAP = average(AP, k)
    return(AA, AR, AP, mAP)

In [13]:
AA, AR, AP, mAP = do_kNN(features_gal, labels_gal, features_query, labels_query, camIds_query, 20, 9)

In [14]:
print('AA = ', AA)
print('AR = ', AR)
print('AP = ', AP)
print('mAP = ', mAP)

AA =  [[0.8321428571428572], [0.8228571428571428], [0.8157142857142857], [0.8064285714285714], [0.7985714285714286], [0.7907142857142857], [0.785], [0.7771428571428571], [0.7678571428571429], [0.7585714285714286], [0.7492857142857143], [0.7371428571428571], [0.7235714285714285], [0.7114285714285714], [0.6928571428571428], [0.6685714285714286], [0.6392857142857142], [0.5964285714285714], [0.5457142857142857], [0.47]]
AR =  [[0.3368120748299323], [0.333198129251701], [0.32852891156462616], [0.32319727891156486], [0.3189047619047621], [0.31254676870748316], [0.30780612244897976], [0.3017899659863948], [0.2959991496598641], [0.28940221088435386], [0.28187244897959196], [0.2725544217687076], [0.262389455782313], [0.2523996598639455], [0.2404676870748298], [0.2250646258503401], [0.2036709183673468], [0.1644481292517004], [0.11708588435374141], [0.06231292517006812]]
AP =  [[0.12857142857142856], [0.1338345864661678], [0.13928571428571507], [0.1450420168067257], [0.15200892857142856], [0.1590

In [67]:
from metric_learn import LMNN

X = features_train
Y = labels_train

lmnn = LMNN(k=4, learn_rate=1e-6)
lmnn.fit(X, Y)

d:\ryan\uni\fyp\venv3\lib\site-packages\metric_learn\lmnn.py:62: UserWarning: use_pca does nothing for the python_LMNN implementation
  warnings.warn('use_pca does nothing for the python_LMNN implementation')


python_LMNN(convergence_tol=0.001, k=4, learn_rate=1e-06, max_iter=1000,
      min_iter=50, regularization=0.5, use_pca=True, verbose=False)

In [68]:
from metric_learn import NCA

nca = NCA(max_iter=10000)
nca.fit(X, Y)

NCA(learning_rate='deprecated', max_iter=10000, num_dims=None, tol=None,
  verbose=False)

In [69]:
from metric_learn import LSML_Supervised

lsml = LSML_Supervised(num_constraints=200)
lsml.fit(X, Y)

LSML_Supervised(max_iter=1000, num_constraints=200, num_labeled=inf,
        prior=None, tol=0.001, verbose=False, weights=None)

In [70]:
from metric_learn import SDML_Supervised

sdml = SDML_Supervised(num_constraints=50, balance_param=0.5, sparsity_param=0.01)
sdml.fit(X, Y)

C:\Users\armig\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\armig\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function pinvh is deprecated; sklearn.utils.extmath.pinvh was deprecated in version 0.19 and will be removed in 0.21. Use scipy.linalg.pinvh instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\armig\AppData\Roaming\Python\Python36\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function graph_lasso is deprecated; The 'graph_lasso' was renamed to 'graphical_lasso' in version 0.20 and will be removed in 0.22.
  warnings.warn(msg, category=DeprecationWarning)


SDML_Supervised(balance_param=0.5, num_constraints=50, num_labeled=inf,
        sparsity_param=0.01, use_cov=True, verbose=False)

In [74]:
from metric_learn import LFDA

X = features_train
Y = labels_train

lfda = LFDA()
lfda.fit(X, Y)

C:\Users\armig\AppData\Roaming\Python\Python36\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1556: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  RuntimeWarning)


LFDA(embedding_type='weighted', k=None, num_dims=None)

In [75]:
from metric_learn import ITML_Supervised

X = features_train
Y = labels_train

itml = ITML_Supervised(num_constraints=200)
itml.fit(X, Y)

ITML_Supervised(A0=None, bounds=None, convergence_threshold=0.001, gamma=1.0,
        max_iter=1000, num_constraints=200, num_labeled=inf, verbose=False)

In [76]:
lfda_gal = lfda.transform(features_gal)
lfda_query = lfda.transform(features_query)

In [77]:
itml_gal = itml.transform(features_gal)
itml_query = itml.transform(features_query)

In [78]:
lsml_gal = lsml.transform(features_gal)
lsml_query = lsml.transform(features_query)

In [79]:
sdml_gal = sdml.transform(features_gal)
sdml_query = sdml.transform(features_query)

In [83]:
nca_gal = nca.transform(features_gal)
nca_query = nca.transform(features_query)

In [84]:
lmnn_gal = lmnn.transform(features_gal)
lmnn_query = lmnn.transform(features_query)


In [85]:
AA, AR, AP, mAP = do_kNN(itml_gal, labels_gal, itml_query, labels_query, camIds_query, 20, 9)

In [86]:
print('AA = ', AA)
print('AR = ', AR)
print('AP = ', AP)
print('mAP = ', mAP)

AA =  [[0.8257142857142857], [0.8214285714285714], [0.8142857142857143], [0.8078571428571428], [0.8035714285714286], [0.795], [0.785], [0.7771428571428571], [0.7692857142857142], [0.7571428571428571], [0.745], [0.7357142857142858], [0.7228571428571429], [0.7078571428571429], [0.6957142857142857], [0.6685714285714286], [0.6357142857142857], [0.6], [0.5464285714285714], [0.4635714285714286]]
AR =  [[0.33455442176870775], [0.3311870748299323], [0.3275178571428575], [0.3228256802721092], [0.31932227891156495], [0.3140382653061228], [0.30728996598639485], [0.30189030612244927], [0.29537755102040836], [0.28798554421768724], [0.28050595238095255], [0.27123724489795925], [0.26157057823129237], [0.24835119047619036], [0.2369268707482992], [0.22100340136054414], [0.20170493197278896], [0.1625136054421766], [0.11677806122448972], [0.06130612244897967]]
AP =  [[0.12782142857142856], [0.1331954887218069], [0.1390079365079374], [0.1450420168067257], [0.1524107142857143], [0.1599047619047624], [0.167

In [87]:
AA, AR, AP, mAP = do_kNN(sdml_gal, labels_gal, sdml_query, labels_query, camIds_query, 20, 9)

In [88]:
print('AA = ', AA)
print('AR = ', AR)
print('AP = ', AP)
print('mAP = ', mAP)
#78.9

AA =  [[0.8192857142857143], [0.8142857142857143], [0.8071428571428572], [0.7978571428571428], [0.7964285714285714], [0.7907142857142857], [0.7842857142857143], [0.7728571428571429], [0.7628571428571429], [0.755], [0.745], [0.73], [0.72], [0.7085714285714285], [0.6921428571428572], [0.6685714285714286], [0.6371428571428571], [0.5957142857142858], [0.5421428571428571], [0.4635714285714286]]
AR =  [[0.3335136054421772], [0.3305119047619051], [0.3259557823129255], [0.3209744897959186], [0.3175561224489798], [0.31212414965986407], [0.3069897959183675], [0.29921513605442196], [0.2929030612244899], [0.28537074829931985], [0.27890561224489807], [0.27055357142857145], [0.26002125850340135], [0.25035969387755097], [0.23833503401360526], [0.22170153061224482], [0.20138860544217663], [0.1633086734693875], [0.11495578231292511], [0.06148554421768717]]
AP =  [[0.12732142857142856], [0.1327819548872204], [0.13825396825396905], [0.14415966386554918], [0.15147321428571428], [0.15876190476190533], [0.1

In [89]:
AA, AR, AP, mAP = do_kNN(lsml_gal, labels_gal, lsml_query, labels_query, camIds_query, 20, 9)

In [90]:
print('AA = ', AA)
print('AR = ', AR)
print('AP = ', AP)
print('mAP = ', mAP)

AA =  [[0.56], [0.5557142857142857], [0.5535714285714286], [0.5478571428571428], [0.5421428571428571], [0.535], [0.5285714285714286], [0.5192857142857142], [0.5157142857142857], [0.5071428571428571], [0.5021428571428571], [0.49642857142857144], [0.4878571428571429], [0.4785714285714286], [0.4692857142857143], [0.455], [0.43], [0.39785714285714285], [0.36142857142857143], [0.31]]
AR =  [[0.1764166666666665], [0.1746964285714284], [0.17334438775510186], [0.17140306122448962], [0.16903911564625831], [0.1666964285714284], [0.16420663265306107], [0.16088605442176857], [0.1584260204081631], [0.15572193877551002], [0.1516530612244896], [0.1480620748299318], [0.1436615646258502], [0.13836139455782304], [0.13263350340136046], [0.12431632653061218], [0.11517006802721083], [0.09674404761904762], [0.07209183673469391], [0.04118282312925174]]
AP =  [[0.06732142857142843], [0.07018796992481281], [0.07349206349206439], [0.07693277310924433], [0.08058035714285715], [0.08476190476190516], [0.0894897959

In [91]:
AA, AR, AP, mAP = do_kNN(lmnn_gal, labels_gal, lmnn_query, labels_query, camIds_query, 20, 9)

In [92]:
print('AA = ', AA)
print('AR = ', AR)
print('AP = ', AP)
print('mAP = ', mAP)

AA =  [[0.8035714285714286], [0.7971428571428572], [0.7928571428571428], [0.7857142857142857], [0.78], [0.775], [0.7671428571428571], [0.7628571428571429], [0.7535714285714286], [0.7442857142857143], [0.7328571428571429], [0.7214285714285714], [0.7071428571428572], [0.69], [0.67], [0.6464285714285715], [0.6185714285714285], [0.5835714285714285], [0.5307142857142857], [0.445]]
AR =  [[0.31665646258503444], [0.3131462585034017], [0.3096913265306126], [0.3042661564625854], [0.3002500000000003], [0.2965850340136057], [0.29179506802721106], [0.286437074829932], [0.2799540816326532], [0.2729107142857143], [0.26464200680272104], [0.2558596938775509], [0.24680527210884337], [0.23688095238095225], [0.2249064625850339], [0.20896173469387747], [0.19024744897959156], [0.15654591836734666], [0.11151190476190469], [0.05898129251700688]]
AP =  [[0.1210357142857144], [0.12597744360902471], [0.13150793650793743], [0.13672268907563292], [0.1433482142857143], [0.15100000000000036], [0.159183673469388], [

In [93]:
AA, AR, AP, mAP = do_kNN(lfda_gal, labels_gal, lfda_query, labels_query, camIds_query, 20, 9)

In [94]:
print('AA = ', AA)
print('AR = ', AR)
print('AP = ', AP)
print('mAP = ', mAP)

AA =  [[0.795], [0.7914285714285715], [0.785], [0.7764285714285715], [0.7714285714285715], [0.7635714285714286], [0.7557142857142857], [0.75], [0.7435714285714285], [0.7321428571428571], [0.7171428571428572], [0.7078571428571429], [0.6914285714285714], [0.67], [0.6492857142857142], [0.6242857142857143], [0.5907142857142857], [0.5585714285714286], [0.5114285714285715], [0.42928571428571427]]
AR =  [[0.3073273809523811], [0.3041190476190478], [0.30071683673469407], [0.2961547619047621], [0.291840986394558], [0.28669387755102066], [0.2811250000000002], [0.27610289115646264], [0.2693673469387756], [0.261670918367347], [0.25311479591836744], [0.24538265306122456], [0.23652380952380955], [0.22621088435374145], [0.21520408163265292], [0.2012857142857142], [0.18164030612244872], [0.15032993197278893], [0.10756547619047621], [0.0567925170068028]]
AP =  [[0.1174285714285717], [0.12233082706767127], [0.12769841269841348], [0.13310924369748148], [0.139375], [0.14600000000000019], [0.15346938775510

In [95]:
AA, AR, AP, mAP = do_kNN(nca_gal, labels_gal, nca_query, labels_query, camIds_query, 20, 9)

In [96]:
print('AA = ', AA)
print('AR = ', AR)
print('AP = ', AP)
print('mAP = ', mAP)

AA =  [[0.8057142857142857], [0.8035714285714286], [0.7985714285714286], [0.7928571428571428], [0.7885714285714286], [0.7821428571428571], [0.7771428571428571], [0.7721428571428571], [0.7621428571428571], [0.7542857142857143], [0.7385714285714285], [0.7264285714285714], [0.7178571428571429], [0.7], [0.6864285714285714], [0.6592857142857143], [0.6307142857142857], [0.5871428571428572], [0.5292857142857142], [0.44642857142857145]]
AR =  [[0.32167602040816345], [0.31842517006802745], [0.31440306122449013], [0.3102636054421772], [0.30447448979591857], [0.29986989795918384], [0.2944353741496599], [0.28834863945578243], [0.28246853741496614], [0.2747380952380954], [0.2671845238095239], [0.25886309523809525], [0.2513647959183673], [0.23992091836734686], [0.2282125850340135], [0.21290901360544198], [0.19341581632653032], [0.1571743197278909], [0.11142091836734695], [0.05915646258503409]]
AP =  [[0.12285714285714283], [0.12800751879699473], [0.13341269841269934], [0.13936974789916243], [0.14526